# Sampling Methods

In large networks, exact inference is often computationally expensive and impractical. Drawing conclusions from information can be uncertain. Sampling helps in approximate inference. We will call the information as 'evidence' and compute probability of 'query' variable which makes up the different conclusions. Computing probability of a query variable (or variables) taking on a value (or set of values) given some evidence is called probabilistic inference. When it is difficult to sample from a probability distribution directly, we use its conditional probabilities. In other words, we can sample a multidimensional probability distribution from its conditional densities.

Given a known probability distribution, a sample will be an assignment of values to each variable in the network. We will then use samples to approximately compute posterior probabilities. Consider the grass sprinkler example. Wet grass can be caused by rain or a sprinkler. Suppose we have observed that the grass is wet. Now consider the additional information that it was cloudy. Does this change your beliefs on how the grass became wet? Let us see.

The nodes Cloudy, Rain, Sprinkler and WetGrass are boolean True(+) or False(-). 

We want to sample values for C. The method used for single variable sampling is that we randomly generate number r between (0,1). If r < 0.5, sample = c+ (c=true), else sample = c- (c=false) 

There are many sampling methods used for approximate inference. We will see four of them here.
* Direct
* Rejection
* Likelihood weighting 
* Gibbs sampling

## Direct Sampling

In direct sampling, we generate samples from a network with no evidence. First, we create a topological order of the variables in the Bayes Network and then sample each variable conditioned on the values of its parents. In the grass sprinkler example, see the below steps for generating a sample of (Cloudy,Sprinkler,Rain,WetGrass) or (C,S,R,W).


<img src="../images/sprinkler.png" style="width: 900px;">


The above sampling process generates samples from the bayes network. We can use the samples to estimate probability of a specific event. An event is defined as the set of values taken by the variables.
Pr[C=t,S=f,R=t,W=t] ≈ [# of samples(t,f,t,t)] / [Total # of samples] becomes exact in large size sample.


### Bayesian Network using pgmpy

Create the network in the above image using pgmpy and verify the probability distribution. Name the model as grass_model. Use the below code for verifying the network.

``` python
for cpd in grass_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
```


In [2]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

cloudy_cpd = TabularCPD(variable='C',
                       variable_card=2,
                       values=[[.5, .5]])

sprinkler_cpd = TabularCPD(variable='S',
                     variable_card=2,
                     values=[[.5, .9],
                           [.5, .1]],
                     evidence=['C'],
                     evidence_card=[2])

rain_cpd = TabularCPD(variable='R',
                     variable_card=2,
                     values=[[.8, .2],
                           [.2, .8]],
                     evidence=['C'],
                     evidence_card=[2])

wetgrass_cpd = TabularCPD(
                variable='W',
                variable_card=2,
                values=[[1, .1, .1, .01],
                        [0, .9, .9, .99]],
                evidence=['S', 'R'],
                evidence_card=[2, 2])

grass_model = BayesianModel()
grass_model = BayesianModel([('C', 'S'),
                             ('C', 'R'),
                             ('S', 'W'),
                             ('R', 'W')])
grass_model.add_cpds(cloudy_cpd, sprinkler_cpd, rain_cpd, wetgrass_cpd)
grass_model.check_model()

# Verify network


True

Verify the probabilities.

In [1]:
for cpd in grass_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

In [2]:
try:
    a=1
    if a == 1:
        ref_assert_var = True
        print('continue')
    else:
        ref_assert_var = False
        print('Please follow the instructions given and use the same variables provided in the instructions. ')
except Exception:
    print('Please follow the instructions given and use the same variables provided in the instructions. ')

continue


## Rejection Sampling

Suppose we have evidence, say Spinkler=True and we want to estimate Pr[Rain=t|Sprinkler=t]. We do direct sampling as described above and then reject all samples that are inconsistent with evidence. Say we took 100 samples of which 50 samples had S=t and 50 samples had S=f. We would reject the samples with S=f and then estimate probability of Rain=t from remaining samples. This is inefficient because we are rejecting most of our samples.


## Likelihood Weighting Sampling

In the above scenario, to estimate Pr[Rain=t | Cloudy=f,WetGrass=t], we will fix the evidence, means we will force the cloudy = true and wetgrass = true and generate samples for Sprinkler and Rain. We generate only those samples that agree with the evidence(here C=f and W=t). We will then weigh them according to the likelihood(probability) of evidence.

To generate a sample of (C,S,R,W), we will set the evidence at (C=f and W=t) and initial weight w=1.
1. C : evidence variable => w = 1 * Pr[C=f] = 0.5
2. S : Sample => Pr[S|C=f] = (.5,.5) => true
3. R : Sample => Pr[R|C=f] = (.2,.8) => false
4. W : evidence variable => w = 0.5 X Pr[W=t|S=t,R=f] = .5 X .9 = .45

We have our first sample as [f,t,f,t] with weight .45.

The limitation of this sampling method is that it gives poor performance if the evidence variables occur later in the sampling order. If evidence variables do not occur early in the order, they cannot be used to influence the samples which would lead to samples with lower weights.

Reference: http://www.cs.cmu.edu/~arielpro/15381f16/slides/781f16-bn.pdf


### Exercise

Consider Pr[Sprinkler=f|Cloudy=t,WetGrass=f]. Compute the likelihood weight for the sample (C,S,R,W)=(t,f,t,f) and assign to variable w.

In [11]:
# w=1

Follow the steps to calculate the likelihood weight of the sample.

In [ ]:
w=1
c=w*.5
w=c*.1

In [2]:
try:
    if w == .05:
        ref_assert_var = True
        print('continue')
    else:
        ref_assert_var = False
        print('Please follow the instructions given and use the same variables provided in the instructions. ')
except Exception:
    print('Please follow the instructions given and use the same variables provided in the instructions. ')

continue


## Generate samples using pgmpy

Use the below code to generate direct, rejection and likelihood weighting samples. Notice that likelihood sample also provides the weight of the sample.

In [10]:
# Direct Sample
from pgmpy.sampling import BayesianModelSampling
inference = BayesianModelSampling(grass_model)
direct = inference.forward_sample(size=2, return_type='recarray')
print("Direct Sample: \n", direct)

# Rejection Sample
from pgmpy.factors.discrete import State
evidence_rej = [State(var='S', state=1)]
rejection = inference.rejection_sample(evidence=evidence_rej, size=3, return_type='dataframe')
print("\n Rejection Sample: \n", rejection)

# Likelihood Weighting Sample
evidence_wt = [State('S', 0),State('W',1)]
likelihood = inference.likelihood_weighted_sample(evidence=evidence_wt, size=2, return_type='recarray')
print("\n Likelihood Weighting Sample: \n", likelihood)


Direct Sample: 
 [(0, 1, 0, 1) (1, 0, 1, 1)]

 Rejection Sample: 
    C  S  R  W
0  0  1  1  1
1  0  1  0  1
2  0  1  0  1

 Likelihood Weighting Sample: 
 [(1, 0, 1, 1,  0.81) (1, 0, 0, 1,  0.  )]


Run the given code to see samples.

In [ ]:
#

In [ ]:
try:
    a=1
    if a == 1:
        ref_assert_var = True
        print('continue')
    else:
        ref_assert_var = False
        print('Please follow the instructions given and use the same variables provided in the instructions. ')
except Exception:
    print('Please follow the instructions given and use the same variables provided in the instructions. ')

## Markov Chain Monte Carlo (MCMC)


Markov process is a process that is capable of being in more than one state, can make transitions among those states, and in which the states available and transition probabilities depend only upon what state the system is currently in. In other words, there is no memory in a Markov process. It is a stochastic process in which future states are independent of past states given the present state. Stochastic process is a consecutive (time component, $t$) set of random quantities defined on some known state(parameter) space $\Theta$.

Monte Carlo is the simulation method used in the MCMC algorithm. Markov Chain Monte Carlo (MCMC) techniques are used for sampling from probability distributions using Markov chain. MCMC methods are used in data modelling for bayesian inference and numerical integration.

Markov Blanket of a node is its parents, children and children's parents. We also assume that nodes are conditionally independent of all other nodes given its Markov Blanket.

The direct, rejection and likelihood weighting sampling methods generate each new sample from scratch. MCMC, on the other hand generates each sample by making a random change to the previous sample. We will now see the Gibbs sampling method which uses the MCMC algorithm.


## Gibbs Sampling

Gibbs sampling procedure starts with initializing all the nodes to a random state. Then sample each state condiditional on all other nodes at their previous states. We will need the find the full conditional of each node.
1. Initialize $C_0, S_0, R_0, W_0$
2. Sample $C_1\mid S_0,R_0,W_0$
3. Sample $S_1\mid C_1,R_0,W_0$
4. Sample $R_1\mid C_1,S_1,W_0$
5. Sample $W_1\mid C_1,S_!,R_1$ 
<br>

New sample is (C_1,S_1,R_1,W_1).

The initial values are typically chosen to be the states with high probability. The steps 2 to 5 are repeated M times and the first B samples are discarded. This is called the burn-in. This is to make our samples closer to the true distribution and less dependent on the starting point. The choice of B is unclear since our samples are not entirely independent and we are unaware of when convergence happens. For high values of M, the value of B does not matter.


### Exercise

Use the below code to generate gibbs samples.


In [3]:
from pgmpy.sampling import GibbsSampling
inferenceG = GibbsSampling(grass_model)
inferenceG.sample(size=2, return_type='dataframe')

,C,S,R,W
0,0,0,1,0
1,1,0,0,0


Run the given code to see samples.

In [ ]:
#

In [ ]:
try:
    a=1
    if a == 1:
        ref_assert_var = True
        print('continue')
    else:
        ref_assert_var = False
        print('Please follow the instructions given and use the same variables provided in the instructions. ')
except Exception:
    print('Please follow the instructions given and use the same variables provided in the instructions. ')